# **configurations**

In [ ]:
!pip install chromadb langchain-community langchain requests together langchain-openai

In [2]:
import os
import json
import re
import chromadb
from chromadb import Client
from chromadb.config import Settings
from google.colab import drive
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
from together import Together
from openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from sentence_transformers import SentenceTransformer, util

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


## constants

In [ ]:
open_ai_key = os.getenv("open_ai_key")
chroma_client_path="/content/drive/MyDrive/pathpik Data/chromadb"
collection_name="all_components"
output_dir = "/content/drive/MyDrive/pathpik Data/outputs/rag-generated-11"

In [5]:
CSS_RESET = """
/* Global Reset and Consistent Base Styles */
* {
  margin: 0;
  padding: 0;
  box-sizing: border-box;
}

html, body {
  width: 100%;
  overflow-x: hidden;
  margin: 0;
  padding: 0;
  scroll-behavior: smooth;
}

#root {
  width: 100%;
  margin: 0;
  padding: 0;
}

.App {
  width: 100%;
  min-height: 100vh;
  margin: 0;
  padding: 0;
  overflow-x: hidden;
  position: relative;
}
"""

CSS_VARIABLES = """
/* Global Design System Variables */
:root {
  --space-xs: 2px;
  --space-sm: 4px;
  --space-1: 8px;
  --space-2: 12px;
  --space-3: 16px;
  --space-4: 20px;
  --space-5: 24px;
  --space-6: 32px;
  --space-7: 40px;
  --space-8: 48px;
  --space-9: 64px;
  --space-10: 80px;

  --text-xs: 0.75rem;
  --text-sm: 0.875rem;
  --text-base: 1rem;
  --text-lg: 1.125rem;
  --text-xl: 1.25rem;
  --text-2xl: 1.5rem;
  --text-3xl: 1.875rem;
  --text-4xl: 2.25rem;
  --text-5xl: 3rem;

  --leading-tight: 1.25;
  --leading-normal: 1.5;
  --leading-relaxed: 1.75;

  --radius-none: 0;
  --radius-sm: 0.125rem;
  --radius-base: 0.25rem;
  --radius-md: 0.375rem;
  --radius-lg: 0.5rem;
  --radius-xl: 0.75rem;
  --radius-2xl: 1rem;
  --radius-full: 9999px;

  --shadow-sm: 0 1px 2px 0 rgba(0,0,0,0.05);
  --shadow-base: 0 1px 3px 0 rgba(0,0,0,0.1), 0 1px 2px 0 rgba(0,0,0,0.06);
  --shadow-md: 0 4px 6px -1px rgba(0,0,0,0.1), 0 2px 4px -1px rgba(0,0,0,0.06);
  --shadow-lg: 0 10px 15px -3px rgba(0,0,0,0.1), 0 4px 6px -2px rgba(0,0,0,0.05);
  --shadow-xl: 0 20px 25px -5px rgba(0,0,0,0.1), 0 10px 10px -5px rgba(0,0,0,0.04);
  --shadow-inner: inset 0 2px 4px 0 rgba(0,0,0,0.06);

  --z-0: 0;
  --z-10: 10;
  --z-20: 20;
  --z-30: 30;
  --z-40: 40;
  --z-50: 50;

  --breakpoint-sm: 640px;
  --breakpoint-md: 768px;
  --breakpoint-lg: 1024px;
  --breakpoint-xl: 1280px;
  --breakpoint-2xl: 1536px;

  --section-gap: 0;
  --component-padding: var(--space-6);
}
"""

In [6]:
CSS_UTILITIES = """
/* Utility Classes */
.flex { display: flex; }
.flex-col { flex-direction: column; }
.items-center { align-items: center; }
.justify-center { justify-content: center; }
.justify-between { justify-content: space-between; }

.w-full { width: 100%; }
.h-full { height: 100%; }
.relative { position: relative; }
.absolute { position: absolute; }
.text-center { text-align: center; }
.font-bold { font-weight: bold; }
.uppercase { text-transform: uppercase; }
.transition-all { transition: all 0.2s ease-in-out; }

.container {
  width: 100%;
  max-width: 1200px;
  margin: 0 auto;
  padding: 0 var(--space-4);
}
"""

In [7]:
first_component_prompt = """
You are a SENIOR React developer and UI/UX expert. Create a PRODUCTION-QUALITY component that establishes excellence.

COMPONENT QUERY: {component_query}
BASE XML: {base_xml}

EXCELLENCE STANDARDS:
**VISUAL EXCELLENCE**:
- Modern, clean, professional design that feels premium
- Thoughtful use of whitespace and visual hierarchy
- Sophisticated color combinations and gradients
- Beautiful typography with proper scale and contrast
- Subtle animations and micro-interactions for polish

**TECHNICAL EXCELLENCE**:
- Performance-optimized React code
- Proper accessibility (ARIA labels, semantic HTML, keyboard navigation)
- Responsive design that works on all devices
- Clean, maintainable code structure
- Error boundaries and loading states where appropriate

**USER EXPERIENCE EXCELLENCE**:
- Intuitive interactions and clear visual feedback
- Smooth animations that enhance usability
- Clear call-to-actions and user guidance
- Fast loading and responsive interactions
- Mobile-first responsive design

MASTER DESIGN SYSTEM (copy exactly for consistency):
```javascript
const theme = {{
  // Colors - Professional and modern palette
  colors: {{
    primary: '#2563eb',      // Professional blue
    secondary: '#10b981',    // Success green
    accent: '#8b5cf6',       // Purple accent
    background: '#ffffff',   // Clean white
    surface: '#f8fafc',      // Light gray surface
    text: '#0f172a',         // Deep text
    textSecondary: '#64748b', // Muted text
    border: '#e2e8f0',       // Subtle borders
    error: '#ef4444',        // Error red
    warning: '#f59e0b',      // Warning amber
    success: '#10b981',      // Success green
  }},

  // Typography scale
  typography: {{
    fontFamily: "'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif",
    fontSize: {{
      xs: '12px',
      sm: '14px',
      base: '16px',
      lg: '18px',
      xl: '20px',
      '2xl': '24px',
      '3xl': '30px',
      '4xl': '36px'
    }},
    fontWeight: {{
      light: 300,
      normal: 400,
      medium: 500,
      semibold: 600,
      bold: 700,
      extrabold: 800
    }}
  }},

  // Spacing system (8px base)
  spacing: {{
    xs: '4px',
    sm: '8px',
    md: '16px',
    lg: '24px',
    xl: '32px',
    '2xl': '48px',
    '3xl': '64px',
    '4xl': '96px'
  }},

  // Border radius
  borderRadius: {{
    sm: '4px',
    md: '8px',
    lg: '12px',
    xl: '16px',
    '2xl': '24px',
    full: '999px'
  }},

  // Shadows
  boxShadow: {{
    sm: '0 1px 2px 0 rgba(0, 0, 0, 0.05)',
    md: '0 4px 6px -1px rgba(0, 0, 0, 0.1)',
    lg: '0 10px 15px -3px rgba(0, 0, 0, 0.1)',
    xl: '0 20px 25px -5px rgba(0, 0, 0, 0.1)',
    '2xl': '0 25px 50px -12px rgba(0, 0, 0, 0.25)'
  }},

  // Animations
  transition: {{
    fast: '150ms ease-in-out',
    normal: '250ms ease-in-out',
    slow: '350ms ease-in-out'
  }}
}};
```

PREMIUM COMPONENT FEATURES TO INCLUDE:
- **Micro-animations**: Hover effects, smooth transitions, loading states
- **Advanced interactions**: Progressive disclosure, smart defaults
- **Visual polish**: Subtle gradients, perfect spacing, elegant typography
- **Accessibility**: Full keyboard navigation, screen reader support
- **Performance**: Optimized images, lazy loading, efficient rendering
- **Responsive**: Mobile-first, touch-friendly, adaptive layouts

MODERN DESIGN PATTERNS:
- Glass morphism effects for cards/modals
- Gradient overlays and backgrounds
- Rounded corners and soft shadows
- Generous whitespace and breathing room
- Clear visual hierarchy with size/weight/color
- Consistent button styles and states

COMPONENT STRUCTURE:
```jsx
import React, {{ useState, useEffect, useCallback }} from 'react';

const ComponentName = () => {{
  const theme = {{ /* Design system above */ }};

  // State management
  const [state, setState] = useState();

  // Performance optimizations
  const optimizedFunction = useCallback(() => {{}}, []);

  // Accessibility and interactions
  useEffect(() => {{
    // Setup keyboard listeners, observers, etc.
  }}, []);

  return (
    <section
      role="main"
      aria-labelledby="section-title"
      style={{{{ ...theme, /* CSS-in-JS styles */ }}}}
    >
      {{/* Premium, accessible, responsive JSX */}}
    </section>
  );
}};

export default ComponentName;
```

Create a PRODUCTION-READY component that showcases modern React and design excellence.
Return only the complete, polished React component code:
"""

In [8]:
components_prompt = """
You are a SENIOR React developer continuing a PREMIUM design system. Maintain EXCELLENCE while ensuring SEAMLESS integration.

COMPONENT QUERY: {component_query}
BASE XML: {base_xml}

ESTABLISHED DESIGN SYSTEM:
{previous_design_system}

PREVIOUS COMPONENT (for visual continuity):
{previous_component_code}

COMPONENT CONTEXT: {component_context}

Rules:
- EXTRACT and USE the exact CSS variables from the previous component
- Match colors, fonts, spacing, and visual style perfectly
- Ensure this component will look cohesive with the previous one
- Use the same design patterns and visual hierarchy

Return a PRODUCTION-QUALITY component that seamlessly continues the established excellence:
"""

In [9]:
package_json_config = {
    "name": "generated-react-app",
    "version": "0.1.0",
    "private": True,
    "dependencies": {
        "react": "^18.2.0",
        "react-dom": "^18.2.0",
        "react-scripts": "5.0.1"
    },
    "scripts": {
        "start": "react-scripts start",
        "build": "react-scripts build",
        "test": "react-scripts test",
        "eject": "react-scripts eject"
    },
    "browserslist": {
        "production": [
            ">0.2%",
            "not dead",
            "not op_mini all"
        ],
        "development": [
            "last 1 chrome version",
            "last 1 firefox version",
            "last 1 safari version"
        ]
    }
}

In [10]:
index_js_template = '''import React from 'react';
import ReactDOM from 'react-dom/client';
import './index.css';
import App from './App';

const root = ReactDOM.createRoot(document.getElementById('root'));
root.render(
  <React.StrictMode>
    <App />
  </React.StrictMode>
);
'''

In [16]:
index_css_template = '''* {
  margin: 0;
  padding: 0;
  box-sizing: border-box;
}

body {
  font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', 'Roboto', sans-serif;
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
}

#root {
  min-height: 100vh;
}
'''

In [17]:
index_html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>Generated React App</title>
</head>
<body>
    <noscript>You need to enable JavaScript to run this app.</noscript>
    <div id="root"></div>
</body>
</html>
"""

In [18]:
readme_template = """
# Generated React App

This React application was generated based on your requirements.

## Generated Components:
{components_list}

## Setup Instructions:

1. Install dependencies:
   ```bash
   npm install
   ```

2. Start development server:
   ```bash
   npm start
   ```

3. Build for production:
   ```bash
   npm run build
   ```

## Project Structure:
- src/components/: Individual React components
- src/App.js: Main app component that imports all components
- src/index.js: React app entry point
- public/index.html: HTML template
"""

In [19]:
app_template = """
Create a main React App component that imports and uses all the generated components.

USER'S ORIGINAL REQUEST: {user_query}

GENERATED COMPONENTS:
{components_info}

 DESIGN SYSTEM:
{design_system}

Create an `App.js` file that:
- Imports all the generated components
- Calls them directly inside the main App component
- Keeps a clean, minimal structure (no extra wrappers or divs unless absolutely necessary)
- Ensures responsive layout by letting each component handle its own styling
- Use a single top-level <div> with basic classes like `min-h-screen bg-background` for consistency

Example structure:
```jsx
import {{ Header }} from './components/Header';
import {{ HeroSection }} from './components/HeroSection';
import {{ SimpleFooter }} from './components/SimpleFooter';

export default function App() {{
  return (
    {{`
    <div className="min-h-screen bg-background">
      <Header />
      <HeroSection />
      <SimpleFooter />
    </div>
    `}}
  );
}}
"""

## client

In [20]:
chroma_client = chromadb.PersistentClient(path=chroma_client_path)
components_collection = chroma_client.get_or_create_collection(name=collection_name)
openai_client = OpenAI(api_key=open_ai_key)

## user query

In [21]:
user_query = """Create a modern, visually engaging landing page targeted at creative professionals.
At the top, include a sleek navigation bar with a logo on the left, simple menu links (with "Team" highlighted), and a contrasting "Join Us" button on the right.
The hero section should feature a warm gradient background, large welcoming headline, and a subtle illustration showing creative collaboration in action.
Just below, highlight the team members section with professional photos, names, roles, and short bios arranged in a neat grid layout to showcase diversity and expertise.
Finish with a minimalist footer that includes social media links, contact email, and a small copyright.
The overall theme should use red as the primary accent color for buttons, highlights, and visual emphasis."""

# **Functions**

In [22]:
# Initialize LLM

def init_langchain_llm(api_key):
    return ChatOpenAI(
        api_key=api_key,
        model_name="gpt-5",
        temperature=0.3,
        max_tokens=100000,

    )

In [23]:
# function to extract subqueries from user query

def extract_subqueries(llm):
    template = """
    Extract UI component queries from this user request.

    User Query: {user_query}

    Return a JSON list of component queries. Each query should be:
    - Fully self-contained with all details from original query
    - Focused on a single UI component
    - Written as a standalone UI request

    Add default header/footer if not mentioned.

    Return only JSON array: ["query1", "query2", ...]
    """

    prompt = PromptTemplate(template=template, input_variables=["user_query"])
    return prompt | llm | StrOutputParser()


In [24]:
# Parses a response to extract component queries. Handles dictionaries, lists, JSON strings, or plain text by returning a cleaned list of query strings.

def parse_component_queries(response_text):
    try:
        # Handle if response is already parsed (dict/list)
        if isinstance(response_text, (list, dict)):
            if isinstance(response_text, list):
                return response_text
            else:
                return [response_text]

        # Try to parse as JSON string
        return json.loads(response_text)
    except:
        # Fallback: split by lines and clean
        if hasattr(response_text, 'split'):
            lines = [line.strip().strip('"').strip("'") for line in response_text.split('\n') if line.strip()]
            return [line for line in lines if len(line) > 10]
        else:
            # If it's not a string, convert to string first
            return [str(response_text)]

In [25]:
# retrieve the svg code of the component from chroma collection

def retrieve_xml(query, comp_collection, metadata_filter=None, n_results=1):

    try:
        query_results = comp_collection.query(
            query_texts=[query],
            n_results=n_results,
            where=metadata_filter if metadata_filter else {}  # apply filter only if provided
        )

        if not query_results["ids"] or not query_results["ids"][0]:
            return None

        # Return first result XML
        xml_code = query_results["metadatas"][0][0].get("xml_code")
        return xml_code

    except Exception as e:
        print(f"Error retrieving XML: {e}")
        return None

In [26]:
# function to extract metadata from query

def extract_metadata(llm):
    template = """
    Extract detailed metadata from the following UI component description.
    Return only valid JSON without any markdown formatting or code blocks.
    Use this exact structure and format:

    {{
      "component_name": "string",
      "device": "string",
      "theme": "string",
      "layout": "string",
      "domain": "string",
      "interactions": ["array", "of", "strings"],
      "accessibility": ["array", "of", "strings"],
      "style": ["array", "of", "strings"],
      "purpose": ["array", "of", "strings"],
      "other": ["array", "of", "strings"]
    }}

    Use lowercase, hyphenated values where appropriate. Arrays should contain relevant string values, or be empty arrays [] if not applicable.

    Description: "{description}"
    """

    prompt = PromptTemplate(template=template, input_variables=["description"])
    chain = prompt | llm | StrOutputParser()

    def _parse_output(description: str):
        try:
            content = chain.invoke({"description": description}).strip()
            metadata = json.loads(content)
        except Exception as e:
            print(f"Parsing failed: {e}")
            metadata = {}

        # ---- Apply defaults ----
        defaults = {
            "component_name": "unspecified",
            "device": "unspecified",
            "theme": "unspecified",
            "layout": "unspecified",
            "domain": "unspecified",
            "interactions": [],
            "accessibility": [],
            "style": [],
            "purpose": [],
            "other": []
        }
        for key, value in defaults.items():
            metadata.setdefault(key, value)

        # ---- Normalize arrays ----
        for key in ["interactions", "accessibility", "style", "purpose", "other"]:
            if isinstance(metadata[key], list):
                metadata[key] = list({s.lower().replace(" ", "-") for s in metadata[key]})

        # ---- Normalize strings ----
        for key in ["component_name", "device", "theme", "layout", "domain"]:
            if key in metadata and isinstance(metadata[key], str):
                metadata[key] = metadata[key].lower().replace(" ", "-")

        return metadata

    return _parse_output

In [27]:
# filter metadata

def clean_metadata(metadata: dict) -> dict:

    allowed_keys = ["component_name", "device", "theme", "layout", "domain"]
    conditions = []

    for key in allowed_keys:
        val = metadata.get(key, None)
        if val and val != "unspecified":
            conditions.append({key: {"$eq": val}})

    return {"$and": conditions} if conditions else {}

In [28]:
# Extract a comprehensive design system from component code

def extract_design_system_comprehensive(component_code):

    # Extract the baseVars object more accurately
    base_vars_pattern = r'const baseVars = \{([^}]+(?:\{[^}]*\}[^}]*)*)\}'
    base_vars_match = re.search(base_vars_pattern, component_code, re.DOTALL)

    # Extract theme objects
    theme_pattern = r'const theme = \{([^}]+(?:\{[^}]*\}[^}]*)*)\}'
    theme_match = re.search(theme_pattern, component_code, re.DOTALL)

    # Extract CSS variables
    css_vars = re.findall(r'--[\w-]+:\s*[^;,}]+', component_code)

    # Extract colors more precisely
    colors = re.findall(r'(?:#[0-9a-fA-F]{6}|#[0-9a-fA-F]{3}|rgb\([^)]+\)|hsl\([^)]+\)|rgba\([^)]+\))', component_code)

    # Extract fonts
    fonts = re.findall(r'fontFamily.*?["\']([^"\']+)["\']', component_code)

    # Extract spacing patterns
    spacing = re.findall(r'(?:padding|margin).*?["\']([^"\']+)["\']', component_code)

    # Extract background patterns
    backgrounds = re.findall(r'background(?:-color)?:\s*["\']?([^"\';\s}]+)["\']?', component_code)

    return {
        'base_vars': base_vars_match.group(1) if base_vars_match else "",
        'theme_object': theme_match.group(1) if theme_match else "",
        'css_variables': list(set(css_vars)),
        'colors': list(set(colors)),
        'fonts': list(set(fonts)),
        'spacing': list(set(spacing)),
        'backgrounds': list(set(backgrounds)),
        'layout_patterns': re.findall(r'(?:display|flexDirection|justifyContent|alignItems).*?["\']([^"\']+)["\']', component_code)
    }

In [29]:
# Create comprehensive CSS for consistent global styles

def create_global_styles_file(components, output_dir):

    # Combine all CSS blocks
    css_content = CSS_RESET + CSS_VARIABLES + CSS_UTILITIES

    css_path = os.path.join(output_dir, "src", "App.css")
    os.makedirs(os.path.dirname(css_path), exist_ok=True)

    with open(css_path, "w", encoding="utf-8") as f:
        f.write(css_content)

    return css_path

# **pipeline**

In [30]:
# Creates the initial component that establishes the design system foundation

def create_first_react_component_chain(llm):
    prompt = PromptTemplate(
        template=first_component_prompt,
        input_variables=["component_query", "base_xml", "original_query"]
    )
    return prompt | llm | StrOutputParser()

In [31]:
# Creates follow-up components that maintain visual continuity with previous ones
# Extracts and reuses design patterns from existing components
# Ensures seamless integration and cohesive user experience across components

def create_subsequent_react_component_chain(llm):
    prompt = PromptTemplate(
        template=components_prompt,
        input_variables=["component_query", "base_xml", "previous_design_system", "previous_component_code", "component_context"]
    )
    return prompt | llm | StrOutputParser()

In [32]:
# Create necessary React project files

def create_react_project_files(output_dir, components):

    # Create package.json
    with open(os.path.join(output_dir, "package.json"), "w") as f:
        json.dump(package_json_config, f, indent=2)

    # Create index.js
    with open(os.path.join(output_dir, "src", "index.js"), "w") as f:
        f.write(index_js_template)

    # Create index.css
    with open(os.path.join(output_dir, "src", "index.css"), "w") as f:
        f.write(index_css_template)

    # Create public/index.html
    public_dir = os.path.join(output_dir, "public")
    os.makedirs(public_dir, exist_ok=True)

    with open(os.path.join(public_dir, "index.html"), "w") as f:
        f.write(index_html_template)

    # Create README.md
    components_list = chr(10).join([f"- {comp['name']}: {comp['query'][:50]}..." for comp in components])
    readme_content = readme_template.format(components_list=components_list)

    with open(os.path.join(output_dir, "README.md"), "w") as f:
        f.write(readme_content)

In [33]:
# Create a main React App component that imports and uses all the generated components.

def create_app_file(llm):
    prompt = PromptTemplate(
        template=app_template,
        input_variables=["user_query", "components_info"]
    )
    return prompt | llm | StrOutputParser()

In [38]:
# Generate React components with enforced design consistency

def generate_chained_react_components(user_query, comp_collection, llm, output_dir):

    # Extract component queries
    extraction_chain = extract_subqueries(llm)
    queries_response = extraction_chain.invoke({"user_query": user_query})
    component_queries = parse_component_queries(queries_response)

    print(f"Extracted {len(component_queries)} React component queries:")
    for i, q in enumerate(component_queries):
        print(f"{i+1}. {q}...")

    # Setup
    metadata_chain = extract_metadata(llm)
    first_component_chain = create_first_react_component_chain(llm)
    subsequent_component_chain = create_subsequent_react_component_chain(llm)

    components_dir = os.path.join(output_dir, "src", "components")
    os.makedirs(components_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, "src"), exist_ok=True)

    generated_components = []
    master_design_system = None
    previous_component_code = ""

    # Generate components with strict consistency
    for i, component_query in enumerate(component_queries):
        print(f"\nGenerating React component {i+1}/{len(component_queries)}...")

        # Get metadata and retrieve base XML
        metadata_full = metadata_chain(component_query)
        metadata_filter = clean_metadata(metadata_full)
        base_xml = retrieve_xml(component_query,comp_collection ,metadata_filter=metadata_filter) or "No template found"

        if i == 0:
            # First component - establish master design system
            react_code = first_component_chain.invoke({
                "component_query": component_query,
                "base_xml": base_xml,
                "original_query": user_query
            })

            # Extract and store the master design system
            master_design_system = extract_design_system_comprehensive(react_code)
            previous_component_code = react_code

        else:
            # Subsequent components - enforce design consistency
            component_context = f"Component {i+1} of {len(component_queries)}. Must maintain exact design consistency with previous components."

            react_code = subsequent_component_chain.invoke({
                "component_query": component_query,
                "base_xml": base_xml,
                "previous_design_system": json.dumps(master_design_system, indent=2),
                "previous_component_code": previous_component_code,
                "component_context": component_context
            })

        # Save component
        filename = f"component_{i+1}.jsx"
        filepath = os.path.join(components_dir, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(react_code)

        generated_components.append({
            "index": i+1,
            "name": filename,
            "query": component_query,
            "metadata": metadata_filter,
            "code": react_code,
            "filepath": filepath,
            "filename": filename
        })

        print(f"Saved: {filename}")

        # Update previous component for context (only for next iteration)
        if i < len(component_queries) - 1:
            previous_component_code = react_code

    return generated_components

In [39]:
def generate_react_website(user_query, comp_collection,llm, output_dir):

    generated_components = generate_chained_react_components(user_query, comp_collection, llm, output_dir)

    if not generated_components:
        print("No components generated!")
        return None

    # create app file
    app_chain = create_app_file(llm)
    components_info = json.dumps([{
        "name": comp["name"],
        "filename": comp["filename"],
        "query": comp["query"][:50] + "...",
        "index": comp["index"],
    } for comp in generated_components], indent=2)

    master_design_system = extract_design_system_comprehensive(generated_components[0]['code'])
    app_code = app_chain.invoke({
        "user_query": user_query,
        "components_info": components_info,
        "design_system": json.dumps(master_design_system, indent=2) # Pass the actual design system data
    })

    # Save App.js
    app_path = os.path.join(output_dir, "src", "App.js")
    with open(app_path, "w", encoding="utf-8") as f:
        f.write(app_code)

    # Create consistent global styles
    css_path = create_global_styles_file(generated_components, output_dir)

    # Create other project files
    create_react_project_files(output_dir, generated_components)


    print(f"\n React app generated!")
    print(f"   Location: {output_dir}")
    print(f"   Components: {len(generated_components)}")
    print(f"   Run 'npm install && npm start' to start development server")

    return {
        "components": generated_components,
        "app_path": app_path,
        "css_path": css_path,
        "output_dir": output_dir,
        "total_components": len(generated_components),
        "consistency_validated": True
    }

# **Run Pipeline**

In [36]:
llm = init_langchain_llm(open_ai_key)

In [40]:
result = generate_react_website(
        user_query=user_query,
        comp_collection=components_collection,
        llm=llm,
        output_dir=output_dir
    )

Extracted 4 React component queries:
1. Design a sleek, modern navigation bar component for a landing page targeted at creative professionals. Place it at the very top. Layout: a logo on the left, simple horizontal menu links between the logo and the right edge (ensure the 'Team' link is visually highlighted/active), and a contrasting 'Join Us' button on the far right. Use red as the primary accent color for buttons, highlights, and visual emphasis. Include accessible hover/focus states, maintain strong color contrast, and ensure full responsiveness across desktop, tablet, and mobile....
2. Create a hero section component for a modern, visually engaging landing page aimed at creative professionals. Use a warm gradient background, a large welcoming headline, and a subtle illustration that depicts creative collaboration in action. Keep the layout clean and spacious with readable, contemporary typography. Apply red as the primary accent color for highlights and emphasis elements while mai

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 43.0MiB/s]


Saved: component_1.jsx

Generating React component 2/4...
Saved: component_2.jsx

Generating React component 3/4...
Saved: component_3.jsx

Generating React component 4/4...
Saved: component_4.jsx

 React app generated!
   Location: /content/drive/MyDrive/pathpik Data/outputs/rag-generated-11
   Components: 4
   Run 'npm install && npm start' to start development server
